In [44]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=49e6f076fec328e2976879d0c761ce1337cdfc95e94a50ffee59ad5a0d8c22a1
  Stored in directory: /Users/madhavi/Library/Caches/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [45]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [46]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [47]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [48]:
df['koi_disposition'].value_counts()

FALSE POSITIVE    3504
CONFIRMED         1800
CANDIDATE         1687
Name: koi_disposition, dtype: int64

In [49]:
df['koi_disposition']=df['koi_disposition'].replace(' ', '_', regex=True)
#df['koi_disposition']

# Select your features (columns)

In [50]:
# Set features. This will also be used as your x values.
# selected_features = df[['names', 'of', 'selected', 'features', 'here']]
# selected_features.head()
X = df.drop("koi_disposition", axis=1)
y = df["koi_disposition"]
print(X.shape, y.shape)

(6991, 40) (6991,)


# Create a Train Test Split

Use `koi_disposition` for the y values

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [52]:
y_train.head()

4002    FALSE_POSITIVE
4246    FALSE_POSITIVE
548     FALSE_POSITIVE
3953    FALSE_POSITIVE
2362         CONFIRMED
Name: koi_disposition, dtype: object

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [53]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [54]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

In [55]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
y_train_categorical

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [56]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [67]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 100)               4100      
_________________________________________________________________
dense_14 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_15 (Dense)             (None, 3)                 303       
Total params: 14,503
Trainable params: 14,503
Non-trainable params: 0
_________________________________________________________________


In [57]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
 - 1s - loss: 0.6063 - accuracy: 0.7051
Epoch 2/60
 - 0s - loss: 0.3716 - accuracy: 0.8095
Epoch 3/60
 - 0s - loss: 0.3519 - accuracy: 0.8161
Epoch 4/60
 - 0s - loss: 0.3375 - accuracy: 0.8341
Epoch 5/60
 - 0s - loss: 0.3324 - accuracy: 0.8419
Epoch 6/60
 - 0s - loss: 0.3281 - accuracy: 0.8417
Epoch 7/60
 - 0s - loss: 0.3201 - accuracy: 0.8491
Epoch 8/60
 - 0s - loss: 0.3245 - accuracy: 0.8455
Epoch 9/60
 - 0s - loss: 0.3154 - accuracy: 0.8499
Epoch 10/60
 - 0s - loss: 0.3135 - accuracy: 0.8537
Epoch 11/60
 - 0s - loss: 0.3085 - accuracy: 0.8560
Epoch 12/60
 - 0s - loss: 0.3008 - accuracy: 0.8646
Epoch 13/60
 - 0s - loss: 0.2990 - accuracy: 0.8638
Epoch 14/60
 - 0s - loss: 0.2966 - accuracy: 0.8634
Epoch 15/60
 - 0s - loss: 0.2984 - accuracy: 0.8602
Epoch 16/60
 - 0s - loss: 0.2933 - accuracy: 0.8674
Epoch 17/60
 - 0s - loss: 0.2912 - accuracy: 0.8676
Epoch 18/60
 - 0s - loss: 0.2875 - accuracy: 0.8720
Epoch 19/60
 - 0s - loss: 0.2834 - accuracy: 0.8766
Epoch 20/60
 - 0s - l

In [58]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.276263186465139, Accuracy: 0.8878718614578247


In [59]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [60]:
print(f"Predicted classes: {prediction_labels}  Actual Labels: {list(y_test[:10])}")
#print(f"Actual Labels: {list(y_test[:10])}")

Predicted classes: ['CANDIDATE' 'FALSE_POSITIVE' 'FALSE_POSITIVE' 'CANDIDATE'
 'FALSE_POSITIVE']  Actual Labels: ['CANDIDATE', 'FALSE_POSITIVE', 'FALSE_POSITIVE', 'CANDIDATE', 'FALSE_POSITIVE', 'CONFIRMED', 'FALSE_POSITIVE', 'FALSE_POSITIVE', 'CONFIRMED', 'CONFIRMED']


## save the model

In [64]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
your_model="deep_learning_model_trained.h5"

filename = 'madhavi.sav'
joblib.dump(your_model, filename)

['madhavi.sav']

In [62]:
# Save the model
model.save("deep_learning_model_trained.h5")

## load the model

In [65]:
# Load the model
from keras.models import load_model
deep_learning_model = load_model("deep_learning_model_trained.h5")

## evaluate the loaded model

In [66]:
model_loss, model_accuracy =deep_learning_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.276263186465139, Accuracy: 0.8878718614578247


## reffered links

In [ ]:
#important links 
#https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
#Voice_Recognition from activity 3
#